# Pseudo-bayesian!

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd

from jax.scipy.stats import norm, expon



In [ ]:
expon.logpdf(3, loc=0)

In [ ]:
import jax.numpy as np
# True values

mu = 1
sigma = 3
y = 1.1

def neg_model_log_prob(params, data):
    mu, log_sigma = params
    sigma = np.exp(log_sigma)
    
    # log-probability of mu under prior.
    mu_log_prob = norm.logpdf(mu, loc=0, scale=10)

    # log-probability of sigma under prior.
    sigma_log_prob = expon.logpdf(sigma)

    # log-likelihood given priors and data
    likelihood_log_prob = np.sum(norm.logpdf(data, loc=mu, scale=sigma))

    # Joint log-likelihood
    return -(mu_log_prob + sigma_log_prob + likelihood_log_prob)


neg_model_log_prob((mu, log_sigma), y)

In [ ]:
from scipy.stats import norm as snorm

In [ ]:
from tqdm.autonotebook import tqdm
data = snorm(loc=1, scale=3).rvs(10000)

neg_model_log_prob((1, 1), data)
# neg_model_log_prob((0, 1), data)

In [ ]:
from jax.experimental.optimizers import adam, exponential_decay, inverse_time_decay


n_steps = 30000
step_size = exponential_decay(0.5, n_steps, 0.01)

In [ ]:
step_size(n_steps)

In [ ]:
mu_start = 10
sigma_start = 15

from jax import grad, jit
from jax.experimental.optimizers import adam, exponential_decay


dlogp = grad(neg_model_log_prob)

opt_init, opt_update, get_params = adam(step_size=0.005, )

init_params = (10., 0.5)

opt_state = opt_init(init_params)

@jit
def step(i, opt_state, y):
    p = get_params(opt_state)
    g = grad(neg_model_log_prob)(p, y)
    return opt_update(i, g, opt_state)


states = []
for i in tqdm(range(n_steps)):
    opt_state = step(i, opt_state, data)
    if i % 10 == 0:
        states.append(opt_state)

In [ ]:
logps = []
for state in tqdm(states):
    logps.append(neg_model_log_prob(get_params(state), data))


params = get_params(opt_state)
neg_model_log_prob(params, data)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(logps)
plt.yscale("log")

In [ ]:
neg_model_log_prob(get_params(states[-1]), data)

In [ ]:
mu, log_sigma = get_params(states[-2050])
mu, np.exp(log_sigma)
# BINGO! WE DID IT!